In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import json
from pprint import pprint as pp
import time


In [2]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import platform
path = None
if platform.system() is 'Windows':
    path = './drivers/geckodriver.exe'
else:
    path = './drivers/geckodriver'
options = Options()
options.headless = True
driver = webdriver.Firefox(options=options,executable_path=path)

In [3]:
url_data = pd.read_csv("./dataset/test.csv")
url_data.head()

,Unnamed: 0,description,image,publishedAt,source.name,source.url,title,url
0,0,One of the major stumbling blocks for Hong Kon...,NaN,2019-09-01 19:54:00 UTC,Quartz,https://qz.com,"Hong Kong’s fast-learning, dexterous protester...",https://qz.com/1698002/hong-kong-protesters-fl...
1,1,"DAVIS COUNTY, Utah (KSTU) — A principal and a ...",NaN,2019-11-01 17:54:00 UTC,WFLA News Channel 8,https://www.wfla.com,Utah elementary student wears Nazi costume in ...,https://www.wfla.com/news/national/utah-princi...
2,2,The principal and a teacher at a Kaysville ele...,NaN,2019-11-01 17:00:00 UTC,The Salt Lake Tribune,https://www.sltrib.com,Utah elementary student wears Nazi costume in ...,https://www.sltrib.com/news/2019/11/01/utah-el...
3,3,A Utah elementary school principal and teacher...,https://images.gnews.io/d01738e949d6a6c9332e4a...,2019-11-05 07:50:00 UTC,YAHOO!,https://news.yahoo.com,"Principal, teacher suspended after student dre...",https://news.yahoo.com/principal-teacher-suspe...
4,4,A Utah elementary school principal and teacher...,NaN,2019-11-05 02:25:00 UTC,huffingtonpost.co.uk,https://www.huffingtonpost.co.uk,"Principal, Teacher Suspended After Student Dre...",https://www.huffingtonpost.co.uk/entry/princip...


In [4]:
def html_decode(s):
    """
    Returns the ASCII decoded version of the given HTML string. This does
    NOT remove normal HTML tags like <p>.
    """
    htmlCodes = (
            ("'", '&#39;'),
            ('"', '&quot;'),
            ('>', '&gt;'),
            ('<', '&lt;'),
            ('&', '&amp;')
        )
    for code in htmlCodes:
        s = s.replace(code[1], code[0])
    return s

In [5]:
def get_page_source(url):
    driver.get(url)
    return driver.page_source

In [6]:
scrapping_url = "https://embed.ly/docs/explore/extract?url="
apikey = "1934f519a63e142e0d3c893e59cc37fe0172e98a"
sharedcount_url = "https://api.sharedcount.com/v1.0/?url={}&apikey={}"
reddit_search = "https://www.reddit.com/domain/{}/search?q={}&sort=top"
colon = '%3A'
fw_slash = '%2F'
def get_sharedcount_url(url):
    return sharedcount_url.format(url,apikey)

def get_fbengagement(url):
    shared_count = json.loads(requests.get(get_sharedcount_url(url)).text)
    if 'Facebook' not in shared_count.keys():
        return {}
    fb = shared_count['Facebook']
    return fb

def scrape_content(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    if soup is None or soup.pre is None:
        return None
    json_text = html_decode(soup.pre.text)
    row = json.loads(json_text)
    if row['content'] is None and row['description'] is None:
        print("Skipping link : ",url)
        return None
    if row['content'] is None and row['description'] is not None :
        row['content'] = row['description']
    row['content']= BeautifulSoup(row['content'], "lxml").text
    row['provider_display'] = row['provider_display'].replace('www.','')
    return row

def scrape_reddit(domain, title):
    query_url = reddit_search.format(domain,title.replace(" ","+"))
#     page = requests.get(query_url).text
    page = get_page_source(query_url)
    soup = BeautifulSoup(page,'html.parser')
    upvote = list()
    for score in soup.findAll('span',{'class':'search-score'}):
        upvote += [int(i) for i in score.text.split() if i.isdigit()] 
    comments = list()
    for score in soup.findAll('a',{'class':'search-comments'}):
        comments += [int(i) for i in score.text.split() if i.isdigit()] 
    return (sum(upvote),sum(comments))

def scrape_news(data):
    for url in data:
        url = url.replace(':',colon)
        url = url.replace('/',fw_slash)
        article = scrape_content(scrapping_url+url)
        if article is None:
            continue
        article.update(get_fbengagement(url))
        article['reddit_upvotes'],article['reddit_comments'] = scrape_reddit(article['provider_display'],article['title'])
        yield article

# for row in scrape_news(url_data['url']):
#     print(row['title'])
    


In [7]:
def create_data(data):
    dataset = list()
    i = 0
    for row in scrape_news(data):
        dataset+=[row]
        print("Adding: ", row['title'])
    return dataset

In [8]:

df = pd.DataFrame(create_data(url_data['url']))
df = df[['language','title','content','description','provider_url','keywords','provider_display','provider_name',
            'total_count','comment_count','share_count','reaction_count','reddit_upvotes','reddit_comments']]


Adding:  Hong Kong's fast-learning, dexterous protesters are stumped by Twitter
Adding:  Utah elementary student wears Nazi costume in Halloween school parade, principal and teacher suspended
Adding:  Utah elementary student wears Nazi costume in Halloween parade. The principal and teacher are now suspended.
Adding:  Principal, teacher suspended after student dresses as Hitler in parade
Adding:  Principal, Teacher Suspended After Student Dresses As Hitler In Halloween Parade
Adding:  Naked Florida Man Accused Of Killing Suspected 'Peeping Tom'
Adding:  Man arrested for killing Peeping Tom in Fort Lauderdale
Adding:  Delray man arrested in 2018 'peeping Tom' killing
Adding:  Man charged with manslaughter after beating peeping Tom to death
Adding:  Florida man accused of beating peeping Tom to death
Adding:  Man accused of killing 'peeping Tom' outside girlfriend's window
Adding:  Florida man arrested for beating peeping Tom to death outside girlfriend's window
Adding:  Naked Florida man

KeyError: 'date'

In [10]:
df['date'] = url_data['publishedAt']
df['date'] = pd.to_datetime(df['date'])
# df['group'] = url_data['group']
df['title_len'] = df['title'].str.split().apply(len)
df['content_len'] = df['content'].str.split().apply(len)
df['description_len'] = df['description'].str.split().apply(len)

df.head()

,language,title,content,description,provider_url,keywords,provider_display,provider_name,total_count,comment_count,share_count,reaction_count,reddit_upvotes,reddit_comments,date,title_len,content_len,description_len
0,en,"Hong Kong's fast-learning, dexterous protester...","\nFor over two months, Hong Kong's protesters ...","For over two months, Hong Kong's protesters ha...",https://qz.com,[],qz.com,Quartz,100,2,59,39,1,0,2019-09-01 19:54:00+00:00,9,833,54
1,en,Utah elementary student wears Nazi costume in ...,\n\nby: David Wells\nPosted: / Updated:\n\n DA...,"DAVIS COUNTY, Utah (KSTU) - A principal and a ...",https://www.wfla.com,[],wfla.com,WFLA,689,320,45,324,2,0,2019-11-01 17:54:00+00:00,14,160,51
2,None,Utah elementary student wears Nazi costume in ...,\n The principal and a teacher at a Kaysville ...,The principal and a teacher at a Kaysville ele...,https://www.sltrib.com,[],sltrib.com,The Salt Lake Tribune,21635,5180,2452,14003,148,32,2019-11-01 17:00:00+00:00,16,217,65
3,en,"Principal, teacher suspended after student dre...",A Utah elementary school principal and teacher...,A Utah elementary school principal and teacher...,https://news.yahoo.com,[],news.yahoo.com,Yahoo,3,0,3,0,0,0,2019-11-05 07:50:00+00:00,10,26,26
4,en,"Principal, Teacher Suspended After Student Dre...",\nA Utah elementary school principal and teach...,A Utah elementary school principal and teacher...,https://www.huffingtonpost.co.uk,[],huffingtonpost.co.uk,HuffPost UK,12,3,2,7,0,0,2019-11-05 02:25:00+00:00,11,488,67


In [11]:
df.to_csv("./dataset/test_meta.csv")

In [12]:
driver.quit()